BETTER AND MORE REALISTIC MOMENTUM STRATEGY

HIGH QUALITY - over a longer period of time, shows a slower and steadier price return

LOW QUALITY - an example would be something that showed 0% gain for 11 months and 10% gain on the 12th... it doesnt have the same guarenteed returns that a high quality would provide over time. (its just a heater....)

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import  mean

IMPORTING OUR LIST OF STOCKS

INIT API KEY

In [ ]:
# SMP 500 CSV
stocks = pd.read_csv("sp_500_stocks.csv")

# api
IEX_CLOUD_API_TOKEN = ''

SORTING ALL STOCKS INTO 2D ARRAY HOLDING A LIST OF 100 STOCKS EACH
[
    [100],
    [100],
    [100],
    [100],
    [100],
]

TURN EACH LIST INTO A STRING 
[
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
]

In [ ]:
# function to split bigger list into smaller lists
def chunks( lst , n  ):
    ''' yeild successive n-sized lists from lst'''
    for i in range( 0 , len(lst) , n ):
        yield lst[i: i + n]

# 2 dimentional list with each inner list containing 100 tickers
symbol_groups = list(chunks( stocks['Ticker'] , 100  ) )
symbol_strings = []

# turns each list into usable string for api call
for i in range( 0 , len(symbol_groups) , 1  ):
    joined_arr =  ",".join(symbol_groups[i])
    symbol_strings.append( joined_arr )


CREATING FINAL OUTPUT PANDAS DATAFRAME

In [ ]:
# creating columns for dataframe
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares To Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'Percentile Score'
]

final_dataframe = pd.DataFrame(columns = hqm_columns)

BATCH API CALLS TO GET PRICES AND STATS

In [ ]:
# create url for each list of 100 stocks
# request data for the sub list
for group_string in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={group_string}&types=advanced-stats,price&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get( batch_api_call ).json()

    # arr of every ticker in sub list
    stock_array_in_sub_list = group_string.split(',')
    for symbol in stock_array_in_sub_list:
        price = data[symbol]['price']
        oneYear_price_return = data[symbol]['advanced-stats']['year1ChangePercent']
        sixMonths_price_return = data[symbol]['advanced-stats']['month6ChangePercent']
        threeMonths_price_return = data[symbol]['advanced-stats']['month3ChangePercent']
        oneMonth_price_return = data[symbol]['advanced-stats']['month1ChangePercent']
        
        #NOTE ***** append to final_dataframe *****
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    price,
                    "N/A",
                    oneYear_price_return,
                    "N/A",
                    sixMonths_price_return,
                    "N/A",
                    threeMonths_price_return,
                    "N/A",
                    oneMonth_price_return,
                    "N/A",
                    "N/A",
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )


# final_dataframe

CALCULATING MOMENTUM PERCENTILES

In [ ]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

# turn none types into zeros
for row in final_dataframe.index:
    for period in time_periods:
        if final_dataframe.loc[row,f'{period} Price Return'] == None:
            # print(final_dataframe.loc[row,f'Ticker'] , period)
            final_dataframe.loc[row,f'{period} Price Return']=0

# take every time period column and the current looped row price return
# check current looped row price return with every other stock price return
# find the rating percent
    # example:
        # apples 1 year price return was placed in the top 60%
for row in final_dataframe.index:
    for period in time_periods:
        change_col = f'{period} Price Return'
        final_dataframe.loc[ row  , f'{period} Return Percentile'  ] = stats.percentileofscore(
            final_dataframe[ change_col ], #all price returns (base calculation from)
            final_dataframe.loc[ row  , change_col  ] #current return (check whole col and find where this number stands [higher or lower?])
        ) / 100


# final_dataframe

CALCULATING HQM SCORE

high quality momentum score - mean of all time periods scores for each stock

"the average of where the stock stands against the other stocks for it's yearly preformance"

In [ ]:
for row in final_dataframe.index:
    momentum_percentiles = []
    # add all percentile scores to momentum_percentiles
    for period in time_periods:
        momentum_percentiles.append( final_dataframe.loc[ row  , f'{period} Return Percentile'  ]    )
    
    # find average of yearly preformance (momentum_percentiles) & append it to the dataframe
    annual_preformance_avg = mean(momentum_percentiles)
    final_dataframe.loc[ row ,  'Percentile Score'  ] = annual_preformance_avg
    
# final_dataframe

FINDING TOP 50 BEST YEARLY PREFORMANCE PERCENTILE AVERAGES

In [ ]:
# Sorting values in DECENDING order and saving it for later use using INPLACE = TRUE
final_dataframe.sort_values( "Percentile Score" , ascending = False , inplace = True   )

# save final_dataframe[:50] as original 
final_dataframe = final_dataframe[:50]

# Resetting index of sorted data_frame
final_dataframe.reset_index(  inplace = True , drop = True )

# # removing Old Index Column
# final_dataframe.drop(labels='index', axis= 'columns', inplace=True, errors='raise')

# final_dataframe

CALCULATING THE NUMBER OF SHARES TO BUY

In [ ]:
# validate user input for portfollio size '$$$'
def user_portfollio_size_input(portfollio_size):
    try:
        val = float(portfollio_size)
        print(f"Your have entered: ${val}\n")
        return True
    except ValueError:
        print("ERROR: Please type a numerical value")
        return False

portfollio_size = input("Please enter value of your portfollio: $")
while not user_portfollio_size_input(portfollio_size):
    portfollio_size = input("Please enter value of your portfollio: $")

In [ ]:
# calculate position size
position_size = float(portfollio_size) / len(final_dataframe.index)
print(f"Your position size is ${'{:,.2f}'.format(position_size)}")

In [ ]:
# update final data frame
for x in range(0 , len(final_dataframe) , 1):
    final_dataframe.loc[x , "Number of Shares To Buy"] =  position_size /  final_dataframe.loc[x , "Price"]


final_dataframe

CREATING EXCEL FILE

In [ ]:
# init writer objects
writer = pd.ExcelWriter('hqm_dataframe.xlsx' , engine='xlsxwriter')
final_dataframe.to_excel(writer , "High Quality Movement" , index = False)

# writer style init
font_color = "ffffff"
background_color = "#0a0a23"

# format style init
string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format' : "$0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
float_format = writer.book.add_format(
    {
        'num_format' : "0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
percent_format = writer.book.add_format(
    {
        'num_format' : "0.00%",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)


# applying formats to .xlsx file
column_formats = {
    'A' : ['Ticker' , string_format],
    'B' : ['Price' , dollar_format],
    'C' : ['Number of Shares To Buy' , float_format],
    'D' : ['One-Year Price Return' , percent_format],
    'E' : ['One-Year Return Percentile' , percent_format],
    'F' : ['Six-Month Price Return' , percent_format],
    'G' : ['Six-Month Return Percentile' , percent_format],
    'H' : ['Three-Month Price Return' , percent_format],
    'I' : ['Three-Month Return Percentile' , percent_format],
    'J' : ['One-Month Price Return' , percent_format],
    'K' : ['One-Month Return Percentile' , percent_format],
    'L' : ['Percentile Score' , percent_format],
}
# loop through formats and apply to excel sheet columns
for col in  column_formats.keys():
    # columns format
    writer.sheets["High Quality Movement"].set_column( f"{col}:{col}" , 25 , column_formats[col][1] )

    # headers format
    writer.sheets["High Quality Movement"].write( f"{col}1" , column_formats[col][0] , column_formats[col][1] )

# SAVE .xlsx
writer.save()
# SAVE INPUTS
with open("user_inputs.txt" , "w") as f:
    f.write( f"Amount Specified: ${float(portfollio_size)}\n"  )
    f.write( f"Amount Invested Into Each Stock: ${position_size}"  )